In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_2751755/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import time
from pyDOE import lhs
import matplotlib
# matplotlib.use('Agg')
# import imageio
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import shutil
import pickle
import scipy.io
import random
import math
from scipy.interpolate import griddata

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.optim import lr_scheduler

import warnings
warnings.filterwarnings("ignore")

In [3]:
import argparse
class Options_AC(object):
    def __init__(self):
        parser = argparse.ArgumentParser()
        parser.add_argument('--use_subnet', default=True, help=' use subnet or not')
        parser.add_argument('--no_cuda', action='store_true', default=False, help='disable CUDA or not')
        parser.add_argument('--dim_hidden', type=int, default=20, help='neurons in hidden layers')
#         parser.add_argument('--dim_hidden', type=int, default=128, help='neurons in hidden layers')
        parser.add_argument('--hidden_layers', type=int, default=4, help='number of hidden layers')
#         parser.add_argument('--hidden_layers', type=int, default=8, help='number of hidden layers')
        parser.add_argument('--lr', type=float, default=1e-3, help='initial learning rate')
        parser.add_argument('--epochs_Adam', type=int, default=15000, help='epochs for Adam optimizer')
#         parser.add_argument('--epochs_Adam', type=int, default=150000, help='epochs for Adam optimizer')
        parser.add_argument('--epochs_LBFGS', type=int, default=1000, help='epochs for LBFGS optimizer')
        parser.add_argument('--newton_iter', type=int, default=100, help='newton_iter for LBFGS optimizer')
        parser.add_argument('--step_size', type=int, default=5000, help='step size in lr_scheduler for Adam optimizer')
        parser.add_argument('--gamma', type=float, default=0.9, help='gamma in lr_scheduler for Adam optimizer')
        
        self.parser = parser

    def parse(self):
        arg = self.parser.parse_args(args=[])
        # arg.cuda = False
        # arg.device = torch.device('cpu')
        arg.cuda = not arg.no_cuda and torch.cuda.is_available()
        arg.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        return arg

args = Options_AC().parse()
print(args.hidden_layers)

def save_model(state, is_best=None, save_dir=None):
    last_model = os.path.join(save_dir, f'last_model.pth')
    torch.save(state, last_model)
    if is_best:
        best_model = os.path.join(save_dir, f'best_model.pth')
#         shutil.copyfile(last_model, best_model)
        shutil.copyfile(best_model, last_model)        

4


In [4]:
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    random.seed(seed)

def grad(outputs, inputs):
    """ compute the derivative of outputs associated with inputs

    Params
    ======
    outputs: (N, 1) tensor
    inputs: (N, D) tensor
    """
    return torch.autograd.grad(outputs, inputs,
                               grad_outputs=torch.ones_like(outputs),
                               create_graph=True)

def activation(name):
    if name in ['tanh', 'Tanh']:
        return nn.Tanh()
    elif name in ['relu', 'ReLU']:
        return nn.ReLU(inplace=True)
    elif name in ['leaky_relu', 'LeakyReLU']:
        return nn.LeakyReLU(inplace=True)
    elif name in ['sigmoid', 'Sigmoid']:
        return nn.Sigmoid()
    elif name in ['softplus', 'Softplus']:
        return nn.Softplus()
    else:
        raise ValueError(f'unknown activation function: {name}')

In [5]:
seed_torch(42)

In [6]:
class Modified_MLP(nn.Module):
    def __init__(self, dim_in, dim_out, dim_hidden, hidden_layers,
                 act_name='tanh', init_name='xavier_normal'):
        super().__init__()
        self.hidden_layers = hidden_layers

        encoder_U = nn.Sequential()
        encoder_U.add_module('fc', nn.Linear(dim_in, dim_hidden, bias=True))
        encoder_U.add_module('act', activation(act_name))
        self.encoder_U = encoder_U

        encoder_V = nn.Sequential()
        encoder_V.add_module('fc', nn.Linear(dim_in, dim_hidden, bias=True))
        encoder_V.add_module('act', activation(act_name))
        self.encoder_V = encoder_V

        model = nn.Sequential()
        model.add_module('fc0', nn.Linear(dim_in, dim_hidden, bias=True))
        model.add_module('act0', activation(act_name))
        for i in range(1, hidden_layers):
            model.add_module(f'fc{i}', nn.Linear(dim_hidden, dim_hidden, bias=True))
            model.add_module(f'act{i}', activation(act_name))
        model.add_module(f'fc{hidden_layers}', nn.Linear(dim_hidden, dim_out, bias=True))

        self.model = model
        if init_name is not None:
            self.init_weight(init_name)

        self.size = self.model_size()

    def init_weight(self, name):
        """初始化网络参数"""

        if name == 'xavier_normal':
            nn_init = nn.init.xavier_normal_
        elif name == 'xavier_uniform':
            nn_init = nn.init.xavier_uniform_
        elif name == 'kaiming_normal':
            nn_init = nn.init.kaiming_normal_
        elif name == 'kaiming_uniform':
            nn_init = nn.init.kaiming_uniform_
        else:
            raise ValueError(f'unknown initialization function: {name}')

        for param in self.parameters():
            if len(param.shape) > 1:
                nn_init(param)
    
    def forward(self, x):
        """模型的正向传播"""
        U = self.encoder_U(x)
        V = self.encoder_V(x)
        for i in range(self.hidden_layers):
            x = self.model[2 * i](x)      # 调用线性层
            x = self.model[2 * i + 1](x)  # 调用激活层
            x = (1 - x) * U + x * V       # 特征融合
        x = self.model[-1](x)             # 调用最后一个线性层得到输出
        return x

    def model_size(self):
        """模型大小"""
        n_params = 0
        for param in self.parameters():
            n_params += param.numel()
        return n_params


## 网络模型(DNN)

In [7]:
class MLP(nn.Module):
    """Deep Neural Network"""

    def __init__(self, L, M, dim_hidden, hidden_layers, dim_out,
                 act_name='tanh', init_name='xavier_normal'):
        super().__init__()
        
        dim_in = M * 2 + 2
        
        model = Modified_MLP(dim_in, dim_out, dim_hidden, hidden_layers)
        
#         model = nn.Sequential()
        
#         model.add_module('fc0', nn.Linear(dim_in, dim_hidden, bias=True))
#         model.add_module('act0', activation(act_name))
#         for i in range(1, hidden_layers):
#             model.add_module(f'fc{i}', nn.Linear(dim_hidden, dim_hidden, bias=True))
#             model.add_module(f'act{i}', activation(act_name))
#         model.add_module(f'fc{hidden_layers}', nn.Linear(dim_hidden, dim_out, bias=True))
            
        self.model = model
        
        self.L = L
        self.M = M
        
        if init_name is not None:
            self.init_weight(init_name)

            
        self.k = nn.Parameter(torch.arange(1, self.M+1).float(), requires_grad=False)
                    
    def init_weight(self, name):
        if name == 'xavier_normal':
            nn_init = nn.init.xavier_normal_
        elif name == 'xavier_uniform':
            nn_init = nn.init.xavier_uniform_
        elif name == 'kaiming_normal':
            nn_init = nn.init.kaiming_normal_
        elif name == 'kaiming_uniform':
            nn_init = nn.init.kaiming_uniform_
        else:
            raise ValueError(f'unknown initialization function: {name}')

        for param in self.parameters():
            if len(param.shape) > 1:
                nn_init(param)
                
    def input_encoding(self, t, x):
        w = 2.0 * math.pi / self.L
        out = torch.hstack([a]) 
        
        return out    
            
    def forward(self, H):
        t = H[:, 0:1]
        x = H[:, 1:2]
        y = H[:, 2:3]
        
        H = self.input_encoding(t, x, y)
        H = self.model(H)
        
        return H
    
    def forward_test(self, x):
        print(f"{'input':<20}{str(x.shape):<40}")
        for name, module in self.model._modules.items():
            x = module(x)
            print(f"{name:<20}{str(x.shape):<40}")
        return x

    def model_size(self):
        n_params = 0
        for param in self.parameters():
            n_params += param.numel()
        return n_params

In [8]:
class DNN(nn.Module):
    """Deep Neural Network"""

    def __init__(self, dim_in, dim_out, dim_hidden, hidden_layers,
                 act_name='tanh', init_name=None):
        super().__init__()
        model = nn.Sequential()

        model.add_module('fc0', nn.Linear(dim_in, dim_hidden, bias=True))
        model.add_module('act0', activation(act_name))
        for i in range(1, hidden_layers):
            model.add_module(f'fc{i}', nn.Linear(dim_hidden, dim_hidden, bias=True))
            model.add_module(f'act{i}', activation(act_name))
        model.add_module(f'fc{hidden_layers}', nn.Linear(dim_hidden, dim_out, bias=True))

        self.model = model
        if init_name is not None:
            self.init_weight(init_name)

    def init_weight(self, name):
        if name == 'xavier_normal':
            nn_init = nn.init.xavier_normal_
        elif name == 'xavier_uniform':
            nn_init = nn.init.xavier_uniform_
        elif name == 'kaiming_normal':
            nn_init = nn.init.kaiming_normal_
        elif name == 'kaiming_uniform':
            nn_init = nn.init.kaiming_uniform_
        else:
            raise ValueError(f'unknown initialization function: {name}')

        for param in self.parameters():
            if len(param.shape) > 1:
                nn_init(param)

    def forward(self, x):
        return self.model(x)

    def forward_test(self, x):
        print(f"{'input':<20}{str(x.shape):<40}")
        for name, module in self.model._modules.items():
            x = module(x)
            print(f"{name:<20}{str(x.shape):<40}")
        return x

    def model_size(self):
        n_params = 0
        for param in self.parameters():
            n_params += param.numel()
        return n_params

In [9]:
model = MLP(L=2.0, M=10, dim_hidden=args.dim_hidden, hidden_layers=args.hidden_layers, dim_out=1)
model.cuda()
args.model=model
model

MLP(
  (model): Modified_MLP(
    (encoder_U): Sequential(
      (fc): Linear(in_features=22, out_features=20, bias=True)
      (act): Tanh()
    )
    (encoder_V): Sequential(
      (fc): Linear(in_features=22, out_features=20, bias=True)
      (act): Tanh()
    )
    (model): Sequential(
      (fc0): Linear(in_features=22, out_features=20, bias=True)
      (act0): Tanh()
      (fc1): Linear(in_features=20, out_features=20, bias=True)
      (act1): Tanh()
      (fc2): Linear(in_features=20, out_features=20, bias=True)
      (act2): Tanh()
      (fc3): Linear(in_features=20, out_features=20, bias=True)
      (act3): Tanh()
      (fc4): Linear(in_features=20, out_features=1, bias=True)
    )
  )
)

## 数据集生成

In [10]:
class Trainset_AC():
    def __init__(self, *args):
        self.args = args
        self.shape = (self.args[1], self.args[0])
        
    def __call__(self):
        return self.data()
    
    def data(self):
        
        Nsd = self.args[0]
        n_ics = self.args[1]
        
        lb = np.array([0.0, 0.0, 0.0])
        ub = np.array([10.0, 1.0, 1.0])
        tx = (ub-lb)*lhs(3, Nsd)+lb
        tx_ad = tx[(tx[:,0]>0.1)&(tx[:,1]>-0.9)&(tx[:,1]<0.9)]
        tx_no_ad = tx[~((tx[:,0]>0.1)&(tx[:,1]>-0.9)&(tx[:,1]<0.9))]
        
        lb_ics = np.array([0.0, 0.0, 0.0])
        ub_ics = np.array([10.0, 1.0, 1.0]) 
        tx_ics = (ub_ics-lb_ics)*lhs(3, n_ics)+lb_ics
        
        u_ics = np.tanh((0.35-np.sqrt((tx[:,[1]]-0.5)**2 + (tx[:,[2]]-0.5)**2))/(2*0.025))
        
        
        tx_ad = torch.from_numpy(tx_ad).float().cuda()
        tx_no_ad = torch.from_numpy(tx_no_ad).float().cuda()
        tx_ics = torch.from_numpy(tx_ics).float().cuda()
        u_ics = torch.from_numpy(u_ics).float().cuda()
        
        return tx_ad, tx_no_ad, tx_ics, u_ics

In [11]:
trainset = Trainset_AC(25600, 512)
args.trainset = trainset
tx_ad, tx_no_ad, tx_ics, u_ics = trainset()
tx_ad.shape

torch.Size([22806, 3])

## 训练

In [12]:
class Unet(DNN):
    """Physics Constrained Neural Networks
    """
    def __init__(self, dim_in=2, dim_out=1, dim_hidden=10, hidden_layers=5,
                 act_name='sigmoid', init_name=None):
        super().__init__(dim_in, dim_out, dim_hidden, hidden_layers,
                         act_name=act_name, init_name=init_name)
        self.model.fc5.bias.requires_grad = False
    def minmaxscaler(self, data):
        mmin = torch.min(data,dim=0)[0]
        mmax = torch.max(data,dim=0)[0]
        data = (data - mmin)/(mmax-mmin)
        output = data/torch.sum(data) * len(data) + 1
        
        return output
    def forward(self, x):
        u = super().forward(x)
        u = self.minmaxscaler(u)
        return u

In [13]:
class Trainer_AC(object):
    def __init__(self, args):
        self.model = args.model
        self.lr = args.lr
        self.gamma = args.gamma

        self.newton_iter = args.newton_iter
        self.step_size = args.step_size
        
        self.model_name = self.model.__class__.__name__
        self.model_path = self._model_path()
        
        self.epochs_Adam = args.epochs_Adam
        self.epochs_LBFGS = args.epochs_LBFGS
        self.optimizer_Adam = optim.Adam(self.model.parameters(), lr=self.lr, betas=(0.9, 0.999))
        self.optimizer_LBFGS = optim.LBFGS(self.model.parameters(),
                                            lr=0.8,
                                            max_iter=self.newton_iter,
                                            tolerance_grad=1.e-5,
                                            tolerance_change=1.e-9)
        self.scheduler = lr_scheduler.ExponentialLR(self.optimizer_Adam, gamma=self.gamma, verbose=True)
        
        # subnet(Unet)
        self.use_subnet = args.use_subnet
        self.Unet = args.Unet
        self.Unet.cuda()
        self.Unet.zero_grad()
        self.optimizer_Adam_Unet = optim.Adam(filter(lambda p: p.requires_grad==True, self.Unet.parameters()), lr=args.lr)    
        self.scheduler_Unet = lr_scheduler.ExponentialLR(self.optimizer_Adam_Unet, gamma=self.gamma, verbose=True)
        
        # data
        self.tx_ad, self.tx_no_ad, self.tx_ics, self.u_ics = args.trainset()
 
        # Logger
        self.loss_log = []
        self.loss_b_log = []
        self.loss_r_log = []
        self.epoch_log = []
        
        self.loss_lbfgs_log = []
        self.loss_b_lbfgs_log = []
        self.loss_r_lbfgs_log = []
        self.epoch_lbfgs_log = []
        
        
    def _model_path(self):
        """Path to save the model"""
        if not os.path.exists('checkpoints'):
            os.mkdir('checkpoints')
        
        path = os.path.join('checkpoints', self.model_name)
        if not os.path.exists(path):
            os.mkdir(path)
        
        return path

    
    def net_r(self, tx):
        tx.requires_grad_(True)

        u = self.model(tx)

        grad_u = grad(u, tx)[0]
        u_t = grad_u[:,[0]]
        u_x = grad_u[:,[1]]
        u_y = grad_u[:,[2]]
        u_xx = grad(u_x, tx)[0][:,[1]]
        u_yy = grad(u_y, tx)[0][:,[2]]

        eps = 0.025
        lam = 10.
        
        residual = u_t - (eps**2 * (u_xx + u_yy) - u**3 + u) * lam

        return residual

    
    def net_u(self, tx):
        u = self.model(tx)
        return u
    
    
    def loss(self, use_ad=False):
        self.r_ad_pred = self.net_r(self.tx_ad)
        self.r_no_ad_pred = self.net_r(self.tx_no_ad)
        self.g_pred = self.net_u(self.tx_ics)

        if use_ad:

            self.r_weights = self.Unet(self.tx_ad)
            loss_r = torch.mean((self.r_weights * self.r_ad_pred)**2) + torch.mean(self.r_no_ad_pred**2)
            loss_b = torch.mean((self.g_pred-self.u_ics)**2)
            loss = loss_r + 100*loss_b

        else:
            loss_r = torch.mean(self.r_ad_pred**2) + torch.mean(self.r_no_ad_pred**2)
            loss_b = torch.mean((self.g_pred-self.u_ics)**2)
            loss = loss_r + 100*loss_b

        return loss, loss_r, loss_b


    def train_Adam(self, mode='0'):
        """
        mode='0'：subnet参与计算，只训练主网络
        mode='1'：subnet参与计算，只训练subnet
        mode='2'：subnet参与计算，主网络和subnet对抗训练
        mode='3'：subnet不参与计算，只训练主网络
        """
        if mode == '0':
            self.optimizer_Adam.zero_grad()

            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=True)
            loss_value.backward()
            self.optimizer_Adam.step()

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()
        
        elif mode == '1':
            self.optimizer_Adam_Unet.zero_grad()
            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=True)
            loss_value.backward()

            for name, parms in self.Unet.named_parameters():
                if parms.requires_grad==True:
                    parms.grad *= -1

            self.optimizer_Adam_Unet.step()        

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()

        elif mode == '2':
            self.optimizer_Adam.zero_grad()
            self.optimizer_Adam_Unet.zero_grad()

            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=True)
            loss_value.backward()
            self.optimizer_Adam.step()

            for name, parms in self.Unet.named_parameters():
                if parms.requires_grad==True:
                    parms.grad *= -1

            self.optimizer_Adam_Unet.step()

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()

        else:
            self.optimizer_Adam.zero_grad()

            loss_value, loss_r_value, loss_b_value = self.loss(use_ad=False)
            loss_value.backward()
            self.optimizer_Adam.step()

            return loss_value.item(), loss_r_value.item(), loss_b_value.item()
        

    def train_LBFGS(self):
        loss_value, loss_r_value, loss_b_value = self.loss()
        
        def closure():
            loss_value, loss_r_value, loss_b_value = self.loss()

            self.optimizer_LBFGS.zero_grad()
            loss_value.backward()

            return loss_value

        self.optimizer_LBFGS.step(closure)
        loss_value = closure()
        
        return loss_value.item(), loss_r_value.item(), loss_b_value.item()
    
    
    def validate(self, epoch):
        self.model.eval()
        loss_value, loss_r_value, loss_b_value = self.loss()
        
        infos = 'Valid   ' + \
                f'Loss:{loss_value:.4e}  ' + \
                f'Loss_r:{loss_r_value:.4e} '
        print(infos)
        self.model.train()
        return loss_value.item()
    
    
    def save_both_model(self, step, epoch):
        # save backbone
        model_state = {
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
        }
        backbone = os.path.join(self.model_path, 'backbone_%s_%d.pth' % (step, epoch))
        torch.save(model_state, backbone)

        # save subnet
        subnet_state = {
            'epoch': epoch,
            'state_dict': self.Unet.state_dict(),
        }    
        subnet = os.path.join(self.model_path, 'subnet_%s_%d.pth' % (step, epoch))
        torch.save(subnet_state, subnet)

    
    def train(self):
        start = time.time()
        best_loss = 1.e10
        
        # 1.只训练主网络
        for epoch in range(5000):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='3')    
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)
                
            if (epoch+1) % 1000 == 0:
                self.save_both_model('step1', epoch+1)
                                
                # data
                self.tx_ad, self.tx_no_ad, self.tx_ics, self.u_ics = args.trainset()
                
        # 2.只训练subnet      
        for epoch in range(2000):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='1')      
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)
                
            if (epoch+1) % 1000 == 0:
                self.save_both_model('step2', epoch+1)
                                
                # data
                self.tx_ad, self.tx_no_ad, self.tx_ics, self.u_ics = args.trainset()
                
                
        # 3.主网络和subnet对抗，一起训练      
        for epoch in range(self.epochs_Adam):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='2')
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            #学习率衰减？
            if (epoch+1) % self.step_size == 0:
                self.scheduler.step()
                self.scheduler_Unet.step()
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)  
                
            if (epoch+1) % 1000 == 0:
                self.save_both_model('step3', epoch+1)
                                
                # data
                self.tx_ad, self.tx_no_ad, self.tx_ics, self.u_ics = args.trainset()
         
        
        # 使用最好的模型
        state_dict = torch.load(f'{self.model_path}/best_model.pth')
        self.model.load_state_dict(state_dict['state_dict'])
            
        # 4.只训练主网络主网络
        for epoch in range(self.epochs_Adam):
            loss_value, loss_r_value, loss_b_value = self.train_Adam(mode='3')
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)
            
            #学习率衰减？
            if (epoch+1) % self.step_size == 0:
                self.scheduler.step()         
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)  
                
            if (epoch+1) % 1000 == 0:
                self.save_both_model('step4', epoch+1) 
                
                # data
                self.tx_ad, self.tx_no_ad, self.tx_ics, self.u_ics = args.trainset()

        # 5.lbfgs只训练主网络主网络    
        for epoch in range(self.epochs_LBFGS):
            loss_value, loss_r_value, loss_b_value = self.train_LBFGS()
            
            self.loss_log.append(loss_value)
            self.loss_r_log.append(loss_r_value)
            self.loss_b_log.append(loss_b_value)
            self.epoch_log.append(epoch)         
            
            if (epoch+1) % 100 == 0:
                running_time = time.time() - start
                start = time.time()
                
                print(f'Epoch #  {epoch+1}/{self.epochs_Adam + self.epochs_LBFGS}' + f'    time:{running_time:.2f}' + '\n' + \
                      f'loss:{loss_value:.2e}, loss_r:{loss_r_value:.2e}, loss_b:{loss_b_value:.2e},')  
                
                valid_loss = self.validate(epoch)
                is_best = valid_loss < best_loss
                state = {
                    'epoch': epoch,
                    'state_dict': self.model.state_dict(),
                    'best_loss': best_loss
                }
                save_model(state, is_best, save_dir=self.model_path)
                
            if (epoch+1) % 100 == 0:
                self.save_both_model('step5', epoch+1) 
                
                # data
                self.tx_ad, self.tx_no_ad, self.tx_ics, self.u_ics = args.trainset()

In [14]:
args.Unet = Unet(dim_in=2)
trainer = Trainer_AC(args)
trainer.train()

Adjusting learning rate of group 0 to 1.0000e-03.
Adjusting learning rate of group 0 to 1.0000e-03.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 7.78 GiB total capacity; 261.22 MiB already allocated; 12.94 MiB free; 270.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 使用最好的模型
state_dict = torch.load(f'{trainer.model_path}/best_model.pth')
model.load_state_dict(state_dict['state_dict'])

In [ ]:
file = f'checkpoints/subnet,{args.epochs_Adam}Adam,{args.epochs_LBFGS}LBFGS'
if os.path.exists(file)==False:
    os.mkdir(file)

In [ ]:
# np.save(f'{file}/loss_r_log.npy',np.array(trainer.loss_r_log))
# np.save(f'{file}/loss_b_log.npy',np.array(trainer.loss_b_log))

In [ ]:
trainer.loss_r_log = np.load(f'{file}/loss_r_log.npy')
trainer.loss_b_log = np.load(f'{file}/loss_b_log.npy')

In [ ]:
import seaborn as sns
with sns.axes_style("darkgrid"):
    fig = plt.figure(figsize=(6, 5))
    plt.rcParams.update({'font.size':16})
    plt.plot(np.arange(len(trainer.loss_r_log)), trainer.loss_r_log, label='$ \mathcal{L}_r$')
    plt.plot(np.arange(len(trainer.loss_b_log)), trainer.loss_b_log, label='$ \mathcal{L}_{ic}$')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.legend()
    plt.savefig(f'{file}/loss.png', dpi=750, bbox_inches = 'tight')

## 预测

In [ ]:
model.cpu();
trainer.Unet.cpu();

In [ ]:
# load the data
data = scipy.io.loadmat('AC.mat')
usol = data['uu']

# Grid
t_star = data['tt'][0]
x_star = data['x'][0]
TT, XX = np.meshgrid(t_star, x_star)

# Reference solution
plt.pcolor(TT, XX, usol, cmap='jet')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')

In [ ]:
# Get trained network parameters
TX = np.hstack((TT.reshape(-1,1), XX.reshape(-1,1)))
TX = torch.from_numpy(TX).double()

model = trainer.model.cpu().double()
u_pred = model(TX).detach().numpy()
u_pred = u_pred.reshape(TT.shape)

error = np.linalg.norm(u_pred - usol) / np.linalg.norm(usol) 
print('Relative l2 error: {:.3e}'.format(error))

In [ ]:
plt.rcParams.update({'font.size':18})

fig = plt.figure(4, figsize=(24, 5))
ax1 = plt.subplot(1, 4, 1)
plt.pcolor(TT, XX, usol, cmap='jet')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title(r'Reference $u(t,x)$')
plt.tight_layout()
ax1.set_aspect(1./ax1.get_data_ratio())

ax2 = plt.subplot(1, 4, 2)
plt.pcolor(TT, XX, u_pred, cmap='jet')
plt.colorbar()
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title(r'Predicted $u(t,x)$')
plt.tight_layout()
ax2.set_aspect(1./ax2.get_data_ratio())

ax3 = plt.subplot(1, 4, 3)
plt.pcolor(TT, XX, np.abs(usol - u_pred), cmap='jet')
cbar = plt.colorbar()
# cbar.formatter.set_powerlimits((0,0))
plt.xlabel('$t$')
plt.ylabel('$x$')
plt.title(r'Absolute error')
plt.tight_layout()
ax3.set_aspect(1./ax3.get_data_ratio())

with sns.axes_style("darkgrid"):
    ax4 = plt.subplot(1, 4, 4)
    plt.plot(np.arange(len(trainer.loss_r_log)), trainer.loss_r_log, label='$ \mathcal{L}_r$')
    plt.plot(np.arange(len(trainer.loss_b_log)), trainer.loss_b_log, label='$ \mathcal{L}_{ic}$')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.legend()
    ax4.set_aspect(1./ax4.get_data_ratio())


plt.savefig(f'{file}/results_ac.png', dpi=400, bbox_inches = 'tight')
plt.show()